<a href="https://colab.research.google.com/github/stevansehn/python-cloud-dataset-regression/blob/main/cloud_dataset_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import root_mean_squared_error
from sklearn.metrics import r2_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import KFold
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#### 1. Execute 10 vezes o 5-folds CV utilizando diferentes valores para o random_state e calcule o R2 médio destas execuções, para o algoritmo de Regressão Linear.

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/RNA_Datasets/cloud.csv')
X = df.drop('Target', axis=1).values
y = df['Target'].values

for i in range(10):
  kf = KFold(n_splits=5, shuffle=True, random_state=i)
  lr_r2 = cross_val_score(LinearRegression(), X, y, cv=kf, scoring='r2')

  print("Resultado para random_state: {:d}".format(i))
  print("R2 médio: ", lr_r2.mean().round(2))

Resultado para random_state: 0
R2 médio:  0.82
Resultado para random_state: 1
R2 médio:  0.82
Resultado para random_state: 2
R2 médio:  0.8
Resultado para random_state: 3
R2 médio:  0.82
Resultado para random_state: 4
R2 médio:  0.86
Resultado para random_state: 5
R2 médio:  0.81
Resultado para random_state: 6
R2 médio:  0.84
Resultado para random_state: 7
R2 médio:  0.86
Resultado para random_state: 8
R2 médio:  0.8
Resultado para random_state: 9
R2 médio:  0.8


#### 2. Utilizando todo o conjunto de dados para induzir um modelo Lasso com alpha=1, identifique quantos atributos tiveram seus respectivos coeficientes zerados.

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/RNA_Datasets/cloud.csv')
X = df.drop('Target', axis=1).values
y = df['Target'].values

model = Lasso()
model.fit(X, y)

fnames = df.columns.values

print("Intercept: ", model.intercept_)
print("Coeficientes das variáveis independentes:")
for variavel, coeficiente in dict(zip(fnames, model.coef_)).items():
    print(f"{variavel}: {coeficiente}")

Intercept:  0.773032970855124
Coeficientes das variáveis independentes:
period: 0.002463817518745851
seeded: -0.0
season: -0.0
NC: 0.1009399492610018
SC: 0.0
NWC: 0.027228530649650674


Três atributos tiveram seus coeficientes zerados.

#### 3. Utilizando todo o conjunto de dados para induzir um modelo de Regressão Linear, identifique os valores dos coeficientes obtidos para cada atributo.

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/RNA_Datasets/cloud.csv')
X = df.drop('Target', axis=1).values
y = df['Target'].values

model = LinearRegression()
model.fit(X, y)

fnames = df.columns.values

print("Intercept: ", model.intercept_)
print("Coeficientes das variáveis independentes:")
for variavel, coeficiente in dict(zip(fnames, model.coef_)).items():
    print(f"{variavel}: {coeficiente}")

Intercept:  -0.1326054533606127
Coeficientes das variáveis independentes:
period: 0.000590225592924412
seeded: 0.033071124027886145
season: 0.0580051846225087
NC: 0.47871521942809175
SC: 0.7103767094223989
NWC: -0.13032922078271758


#### 4. Divida o dataset em 80%, 20% e utilizando as mesmas partições compare o desempenho dos algoritmos Regressão Linear, Lasso e Ridge usando a medida RMSE.

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/RNA_Datasets/cloud.csv')
X = df.drop('Target', axis=1).values
y = df['Target'].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

lr = LinearRegression()
lr.fit(X_train, y_train)
lr_pred = lr.predict(X_test)
lr_rmse = root_mean_squared_error(y_test, lr_pred)

ridge = Ridge(alpha=1)
ridge.fit(X_train, y_train)
ridge_pred = ridge.predict(X_test)
ridge_rmse = root_mean_squared_error(y_test, ridge_pred)

lasso = Lasso(alpha=1)
lasso.fit(X_train, y_train)
lasso_pred = lasso.predict(X_test)
lasso_rmse = root_mean_squared_error(y_test, lasso_pred)

print("Resultados: ")
print("Regressão Linear: ", lr_rmse)
print("           Lasso: ", lasso_rmse)
print("           Ridge: ", ridge_rmse)

Resultados: 
Regressão Linear:  0.6398333853418955
           Lasso:  0.9642593311555788
           Ridge:  0.6474327641021329


#### 5. Utilizando a técnica de Leave-one-out calcule o valor do erro médio absoluto (MAE) para o algoritmo de Regressão Linear

Obs: A documentação do Scikit Learn diz:



> Note: LeaveOneOut() is equivalent to KFold(n_splits=n) and LeavePOut(p=1) where n is the number of samples.
>
> https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.LeaveOneOut.html


Então ao invés de usar o método LeaveOneOut() eu usei o Kfold com n = número de amostras

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/RNA_Datasets/cloud.csv')
X = df.drop('Target', axis=1).values
y = df['Target'].values

kf = KFold(n_splits=X.shape[0], shuffle=True, random_state=0)

model = LinearRegression()
predictions = cross_val_predict(model, X, y, cv=kf)

mae = mean_absolute_error(y, predictions)

print("Resultado: ")
print("MAE: ", mae)

Resultado: 
MAE:  0.261532090957942
